# Quantum Mech III Homework 4
Michael Lamparski

7 October 2016

$\newcommand\up\uparrow$
$\newcommand\down\downarrow$
$\newcommand\Basis{\mathbf{B}}$
$\newcommand\BUp{\mathbf{B}_\up}$
$\newcommand\BDown{\mathbf{B}_\down}$
$\newcommand\NUp{N_\up}$
$\newcommand\NDown{N_\down}$
$\newcommand\abs[1]{\left|#1\right|}$
$\newcommand\paren[1]{\left(#1\right)}$
$\newcommand\create[1]{a^\dagger_{#1}}$
$\newcommand\bra[1]{\left<#1\right|}$
$\newcommand\ket[1]{\left|#1\right>}$

Note: Most of this was written over the weekend for the $N=4$ problem, with small adaptions added for $N=3$ afterwards.

In [1]:
{-# LANGUAGE DeriveGeneric BangPatterns #-}

import Data.Tuple(swap)
import Data.Function(on)

import Data.Map.Strict(Map)
import qualified Data.Map.Strict as Map
import Data.Vector(Vector,(!),(//)) -- package vector
import qualified Data.Vector as Vector 
import qualified Data.List as List

import qualified Data.Permute as Permute -- package permutation

import Data.Foldable
import Data.Monoid

import Control.Monad
import Control.Exception
import Debug.Trace
import Text.Printf(printf)

import System.IO

# Vacuum expectation of a product of fermion operators

Here is an algorithm to compute the matrix element $\left<0\right|abc\dots xyz\left|0\right>$, where $\left|0\right>$ is the vacuum, and $abc\dots xyz$ are a product of fermion annihilation and creation operators. The algorithm was devised on a whim, and is not particularly efficient or well adapted to the specific problem at hand.

First, primitive types for fermion operators (and products thereof) are defined:

In [2]:
data Fermion i = Annihilate i
               | Create     i
               deriving (Eq, Show)

type Operator i = [Fermion i]
type Basis i = Vector (Operator i)

conjOne :: Fermion i -> Fermion i
conjOne (Annihilate i) = (Create i)
conjOne (Create i)     = (Annihilate i)

conj :: Operator i -> Operator i
conj = reverse . map conjOne

number :: i -> Operator i
number i = [Create i, Annihilate i]

fermionLabel :: Fermion i -> i
fermionLabel (Annihilate i) = i
fermionLabel (Create i)     = i

To compute $\left<0\right|abc\dots xyz\left|0\right>$, we basically "sort" the elements into normal form.

Either **all of them will disappear in the process,** producing a 1; or **at least one of them will survive,** producing 0. Bubble sort actually makes itself useful for once, because it performs
every individual swap that might have a commutation relation.

*__Note:__ one might suspect a single iteration of bubble sort is enough,
as typically one iteration brings the max element to the end,
which is either an annihilator* (which gets applied to the vacuum ket)
*or a creator* (in which case it being the max implies ALL are creators).
*However, this is not actually true in our case. Because some elements disappear each
iteration, it is possible for the max element to be one that we previously
skipped over when a larger element still existed.*

In [3]:

-- Ordering for the (<), (<=), etc. operators that produces normal form.
-- Creation operators are sorted ascendingly by index while annihilators
--  are sorted descending by index, so that the preferred ordering looks
--  like this:  (a+) (b+) (c+) c b a
instance (Ord i) => Ord (Fermion i) where
    (Create     a) `compare` (Create     b) = a `compare` b
    (Create     a) `compare` (Annihilate b) = LT
    (Annihilate a) `compare` (Create     b) = GT
    (Annihilate a) `compare` (Annihilate b) = b `compare` a

applyNTimes :: Int -> (a -> a) -> (a -> a)
applyNTimes n f = foldr (.) id $ replicate n f

-- compute  <0| operator |0>
vacuumValue :: (Ord i, Num x) => Operator i -> x
vacuumValue = afterBubble . bubble where

    -- start with prefactor 1 and accumulated total of 0.
    -- (NOTE: the accumulated total tracks extra terms produced by the
    --        a(a+) = 1 - (a+)a  commutation relation, but it isn't tested
    --        very well since they always come out to zero in the Hubbard model)
    bubble xs = applyNTimes (length xs) (iter []) (1, 0, xs) where

        -- a single full-list bubble sort iteration
        iter done (c, tot, (x0:x1:xs))

            -- a(a+) = 1 - (a+)a
            -- computation branches off to compute the (a+)a term,
            --    after which we continue with the the 1 term.
            | specialPair x0 x1 =
                 let term2 = vacuumValue (reverse done ++ [x1,x0] ++ xs)
                 -- when this assertion fails, it will be necessary to test
                 --  the code that tracks "tot" more rigorously
                 in  assert (term2 == 0) $
                     iter done (c, tot - c * term2, xs)

            -- square of fermion operator is zero (fast path)
            | x0 == x1 = iter []        (0, tot, [])
            -- unrelated terms anticommute
            | x0 >  x1 = iter (x1:done) (-c, tot, (x0:xs))
            | x0 <  x1 = iter (x0:done) ( c, tot, (x1:xs))

        -- We can take a fast track out for an annihilator on the vacuum state.
        iter done (c, tot, [Annihilate _]) = (0, tot, [])
        -- Prepare for the next iteration.
        iter done (c, tot, [x]) = (c, tot, reverse (x:done))
        iter done (c, tot, [ ]) = (c, tot, reverse    done )

        specialPair (Annihilate i) (Create k) | i == k = True
        specialPair _ _ = False

    -- no operators remain.  We have <0|0> times a constant
    afterBubble (c, tot, []) = tot + c 
    -- operators remain; because they are in normal form, this must be zero.
    afterBubble (c, tot, xs) = tot


# Formulating the Hubbard model

## Basis states


We have a half-filled Hubbard model with $\NUp$ spin-up electrons, $\NDown$ spin-down electrons, and $N=\NUp + \NDown$ ions, each providing a localized spin up state and a spin down state.  Because electrons are fermions, no two can occupy the same state, hence we must choose a *combination* of the available localized states to host electrons.

Because we have a fixed number of electrons of each spin, and because the set of states available to each spin are mutually exclusive, we can choose the up spins and the down spins independently. I.e. the basis set can be written as $\Basis=\BUp \times \BDown$, a direct product of individual bases for each spin:


\begin{equation}
\abs\Basis = 
\abs\BUp \cdot \abs\BDown = 
\underset{\text{(in general)}}{\binom N\NUp \binom N\NDown} =
\underset{\text{(half-filled model)}}{\binom N\NUp \binom N{N-\NUp}} =
{\binom N\NUp}^2
\end{equation}

where $\binom mn = \frac{m!}{n!(m-n)!}$ is a binomial coefficient.

For $\NUp=2, \NDown=1$, the **total rank of our basis** is ${\binom32}^2=3^2=9$, consisting of the following states (a cartesian product):

\begin{equation}
\ket{\psi} = \ket{\psi_\up} \otimes \ket{\psi_\down},\qquad
\ket{\psi_\up} \in \begin{Bmatrix}
    \create{1\up} \create{2\up} \ket0 \\
    \create{1\up} \create{3\up} \ket0 \\
    \create{2\up} \create{3\up} \ket0
\end{Bmatrix},~~
\ket{\psi_\down} \in \begin{Bmatrix}
    \create{1\down} \ket0 \\
    \create{2\down} \ket0 \\
    \create{3\down} \ket0
\end{Bmatrix}
\end{equation}

The following generates such a basis, and was used when I looked at the N=4 basis:

In [4]:
data Spin i = Up i | Down i deriving (Eq, Ord, Show)


-- combinations from a list, one of haskell's missing batteries...
combinations :: Int -> [a] -> [[a]]
combinations 0 _ = [[]]
combinations n xs = [ xs !! i : x | i <- [0..(length xs)-1] 
                                  , x <- combinations (n-1) (drop (i+1) xs) ]

-- for N_up = N_down
productBasisN :: Int -> Basis (Spin Int)
productBasisN n | odd n     = error "odd number"
                | otherwise = productBasisNUD n (n `div` 2) (n `div` 2)

-- allows independent N, N_up, and N_down counts.
productBasisNUD :: Int -> Int -> Int -> Basis (Spin Int)
productBasisNUD n u d = Vector.fromList
                           -- behold, the beautiful simplicity of a direct product
                           [ Create <$> (map Up ups ++ map Down dns)
                           | ups <- combinations u [1..n]
                           , dns <- combinations d [1..n]
                           ]
putStrLn "Basis operators in the direct product formulation"
mapM_ print $ productBasisNUD 3 2 1

Basis operators in the direct product formulation

[Create (Up 1),Create (Up 2),Create (Down 1)]
[Create (Up 1),Create (Up 2),Create (Down 2)]
[Create (Up 1),Create (Up 2),Create (Down 3)]
[Create (Up 1),Create (Up 3),Create (Down 1)]
[Create (Up 1),Create (Up 3),Create (Down 2)]
[Create (Up 1),Create (Up 3),Create (Down 3)]
[Create (Up 2),Create (Up 3),Create (Down 1)]
[Create (Up 2),Create (Up 3),Create (Down 2)]
[Create (Up 2),Create (Up 3),Create (Down 3)]

However, this differs from the basis requested on the HW in two ways:

* We sorted by spin first, but the HW requests to sort by site. (this is easily overcome)
* We produced basis elements by a lexical ordering.  The order of states on the HW is fixed
  to group together states which are physically similar.
  
The second in particular is a nice and desirable quality, but it takes more than a simple fix to implement.  Rather than a direct product, it is better described as a \[disjoint\] union: **The states where every ion has an electron, followed by the states where one has two.**

These two groups can each be described as permutations of some representative basis element ($\ket{\psi_1}$ and $\ket{\psi_4}$ on the homework).

In [5]:
----------------------------
-- basis order

data SiteClass = Zero | Plus | Minus | Two deriving (Show, Eq)
data HWState = HWState { hwStateClasses :: [SiteClass] } deriving (Show, Eq)
siteSpins :: SiteClass -> [a -> Spin a]
siteSpins Zero  = []
siteSpins Plus  = [Up]
siteSpins Minus = [Down]
siteSpins Two   = [Up, Down]
siteSpinsAt :: a -> SiteClass -> [Spin a]
siteSpinsAt label = map ($ label) . siteSpins

-- permutations arranged by the ordering scheme used on the hw, insofar as I
-- can surmise, which is to choose the location of the first value, then the
-- location of the 2nd, etc.
-- I.e. the *target* index lists are lexicographically ordered.
permutationsALaGiedt :: [a] -> [[a]]
permutationsALaGiedt xs = iter $ Just $ Permute.permute (length xs) where
    xvec = Vector.fromList xs
    getPerm perm = Vector.toList $ Vector.backpermute xvec
                 -- backpermute needs source indices,
                 --  so invert our target indices
                 $ Vector.fromList $ Permute.elems $ Permute.inverse perm
    iter (Just perm) = getPerm perm:iter (Permute.next perm)
    iter Nothing     = []

buildBasis :: [[SiteClass]] -> [Operator (Spin Int)]
buildBasis = map classesToBasisElement . buildBasisClasses

buildBasisClasses :: [[SiteClass]] -> [HWState]
buildBasisClasses = map HWState . concatMap (List.nub . permutationsALaGiedt)

classesToBasisElement :: HWState -> Operator (Spin Int)
classesToBasisElement = map Create
                      . concat . zipWith siteSpinsAt [1..] . hwStateClasses

With all that infrastructure in place, how about we finish the job?

### Enumerate the basis states in the $m_i=\{0,\pm1,2\}$ format

In [6]:
padL :: a -> Int -> [a] -> [a]
padL fill n xs = replicate (n - length xs) fill ++ xs
padR :: a -> Int -> [a] -> [a]
padR fill n xs = xs ++ replicate (n - length xs) fill

hw4Groups  = [[Plus, Plus, Minus], [Two, Plus, Zero]]
hw4Classes = buildBasisClasses hw4Groups
hw4Basis   = Vector.fromList $ buildBasis hw4Groups
mapM_ putStrLn $ List.zipWith3 (\n a b -> "Ket " ++ show n ++ "   "
                                       ++ padR ' ' 20 (show a) ++ show b)
                 [1..]
                 (hwStateClasses <$> hw4Classes)
                 (map fermionLabel <$> toList hw4Basis)

Ket 1   [Plus,Plus,Minus]   [Up 1,Up 2,Down 3]
Ket 2   [Plus,Minus,Plus]   [Up 1,Down 2,Up 3]
Ket 3   [Minus,Plus,Plus]   [Down 1,Up 2,Up 3]
Ket 4   [Two,Plus,Zero]     [Up 1,Down 1,Up 2]
Ket 5   [Two,Zero,Plus]     [Up 1,Down 1,Up 3]
Ket 6   [Plus,Two,Zero]     [Up 1,Up 2,Down 2]
Ket 7   [Zero,Two,Plus]     [Up 2,Down 2,Up 3]
Ket 8   [Plus,Zero,Two]     [Up 1,Up 3,Down 3]
Ket 9   [Zero,Plus,Two]     [Up 2,Up 3,Down 3]

where e.g. the first row is to be read as $\ket{\psi_1}\equiv\ket{1,1,-1} = a^\dagger_{1\up}a^\dagger_{2\up}a^\dagger_{3\down} \ket{0}$.

## Hamiltonian

Now let's define $\mathbf T$ and $\mathbf U$ matrices straightforwardly from the Hamiltonian.

In [7]:
pairs :: [a] -> [(a,a)]
pairs xs@(a:b:_) = (a,b):pairs (tail xs)
pairs _ = []

matricesN :: (Num a) => Int -> Basis (Spin Int) -> ([[a]], [[a]])
matricesN nIon basis = (getMatrix (-1) tTerms, getMatrix 1 uTerms) where
    -- Funny; I can't seem to come up with a formulation for this that doesn't
    -- have at least one edge case.  Something "magical" seems to happen
    -- between 2 and 3 ions that suddenly doubles the number of interactions.
    interactionPairs 0 = []
    interactionPairs 1 = [(1,1)]
    interactionPairs 2 = [(1,2), (2,1)]
    interactionPairs n = [id,swap] <*> (n,1):pairs [1..n]

    tTerms = [ [Create (spin k), Annihilate (spin i)]
             | (i, k) <- interactionPairs nIon
             , spin   <- [Up, Down]
             ]

    uTerms = [ number (Up i) ++ number (Down i) | i <- [1..nIon] ]

    getElement bra ket mat = vacuumValue $ (conj bra) ++ mat ++ ket
    getMatrix c f = [ [ (c*) $ sum $ getElement bra ket <$> f
                      | ket <- toList basis ]
                    | bra <- toList basis ]

-- Convenience functions for obtaining one at a time.
-- Note that because Haskell is lazy (and because there are no data
--  dependencies between the matrices), calling both of these functions
--  functions does not incur any significant cost compared to calling
--  `matricesN` once; each matrix will still only be computed once.
tMatrixN n basis = fst $ matricesN n basis
uMatrixN n basis = snd $ matricesN n basis

Test on $N=2$:

In [8]:
do
    putStrLn "\nN=2 basis"
    mapM_ print $ productBasisN 2
    putStrLn "\nN=2 t coefficients"
    mapM_ print $ tMatrixN 2 (productBasisN 2)
    putStrLn "\nN=2 U coefficients"
    mapM_ print $ uMatrixN 2 (productBasisN 2)


N=2 basis
[Create (Up 1),Create (Down 1)]
[Create (Up 1),Create (Down 2)]
[Create (Up 2),Create (Down 1)]
[Create (Up 2),Create (Down 2)]

N=2 t coefficients
[0,-1,-1,0]
[-1,0,0,-1]
[-1,0,0,-1]
[0,-1,-1,0]

N=2 U coefficients
[1,0,0,0]
[0,0,0,0]
[0,0,0,0]
[0,0,0,1]

This produced and matrices $\mathbf{T}$ and $\mathbf{U}$ of prefactors satisfying

$$ \hat H = t\, \mathbf{T} + U \mathbf{U} $$

Notice that the t matrix **includes the -1 factor from the Hamiltonian**.

Now let's peek at the $N=3$ matrices.

*(Note added at the eleventh hour: While this notebook never shows the Hamiltonian explicitly, the full expansion was verified to contain 12 terms for $t$ and three (3) terms for $U$.  Each of the six localized states gets two $t$ terms where it is the creation operator.  These terms pair it with the annihilation operators for the states of equal spin at each neighboring site)*

In [9]:
putStrLn "\nN=3 t coefficients"
mapM_ print $ tMatrixN 3 hw4Basis

putStrLn "\nN=3 u coefficients"
mapM_ print $ uMatrixN 3 hw4Basis


N=3 t coefficients

[0,0,0,1,0,-1,0,-1,1]
[0,0,0,0,-1,1,-1,1,0]
[0,0,0,-1,1,0,1,0,-1]
[1,0,-1,0,-1,1,0,0,0]
[0,-1,1,-1,0,0,0,1,0]
[-1,1,0,1,0,0,-1,0,0]
[0,-1,1,0,0,-1,0,0,1]
[-1,1,0,0,1,0,0,0,-1]
[1,0,-1,0,0,0,1,-1,0]


N=3 u coefficients

[0,0,0,0,0,0,0,0,0]
[0,0,0,0,0,0,0,0,0]
[0,0,0,0,0,0,0,0,0]
[0,0,0,1,0,0,0,0,0]
[0,0,0,0,1,0,0,0,0]
[0,0,0,0,0,1,0,0,0]
[0,0,0,0,0,0,1,0,0]
[0,0,0,0,0,0,0,1,0]
[0,0,0,0,0,0,0,0,1]

There are a bunch of $+t$ terms hiding in the $t$ matrix!  Specifically, there are 18 of them:

In [10]:
counts :: (Ord k) => [k] -> Map k Int
counts = Map.fromListWith (+) . map (flip (,) 1)

counts $ concat $ tMatrixN 3 hw4Basis

fromList [(-1,18),(0,45),(1,18)]

Could we improve our basis so that all jumping interactions are $-t$, reflecting the sign in the Hamiltonian.

# A better basis?

Since relative phases between orthogonal states is irrelevant, **we could have freely chosen the sign of any of our basis elements.**  Negating a basis element can also be understood as taking an odd permutation of its operators (due to the anti-commutation relation $\{a_i^\dagger, a_j^\dagger\}=0$).

Assign to each basis element a Boolean variable $x_i$ which is 1 if the basis element is negated, and 0 otherwise.  Let $=$ and $\ne$ represent the logical IFF and XOR operators, which evaluate to 1 if and only if the sum of the two operands is even and odd, respectively.  Then:

* If $\left<i\right|H\left|j\right>=+t$, we require $(x_i\ne x_j)$.
* If $\left<i\right|H\left|j\right>=-t$, we require $(x_i = x_j)$.

This formulates the problem as a **Boolean satisfiability problem.** For this, there is a haskell package `incremental-sat-solver`, maintained by Sebastian Fischer:

In [11]:
import Data.Boolean.SatSolver(SatSolver, Boolean(..))
import Data.Boolean.SatSolver(assertTrue, lookupVar, solve, newSatSolver)

-- Example of how ridiculously generic Haskell can sometimes be:
--
-- Mirroring incremental-sat-solver's own API, the return value of this can be
--  any arbitrary MonadPlus type (which are types that can represent "failure").
-- Depending on how this function is used, on failure it will do anything from
--  returning an empty list, to returning Nothing, to throwing an exception,
--  all within the confines of a strong type system that permits no ambiguity.
findNiceBasis :: (Num a, Eq a, MonadPlus m)
              => [[a]] -- ^ input matrix
              -> m [a] -- ^ output sign factors
findNiceBasis = findNiceBasisWithTarget (-1)

findNiceBasisWithTarget :: (Num a, Eq a, MonadPlus m)
              => a     -- ^ sign
              -> [[a]] -- ^ input matrix
              -> m [a] -- ^ output sign factors
findNiceBasisWithTarget target mat = result where

    -- Data.Boolean.SatSolver.Boolean constructors
    iff a b = (a :&&: b) :||: (Not a :&&: Not b)
    xor a b = Not (a `iff` b)
    -- convert one triangular half to (x,i,j) format
    xij = [ (x,i,j) | (i,row) <- zip [0..] mat, (j,x) <- zip [i..] (drop i row) ]
    -- boolean conditions that must be satisfied
    conditions = map foo xij where
        foo (0,i,j) = Yes -- These don't matter.
        foo (x,i,j)
           | x == -target = Var i `xor` Var j -- aim to negate these elements,
           | x ==  target = Var i `iff` Var j -- but DON'T negate these.
        

    result = do
        let solver = newSatSolver
        -- require each individual condition in turn
        solver <- foldM (flip assertTrue) solver conditions
        -- let the solver guess a solution
        solver <- solve solver
        -- read the solution.  lookupVar returns Nothing for boolean variables
        --   which are not sufficiently constrained; in this case, we prefer
        --   to not negate the kets
        let vars = [0..length mat - 1]
        let solution = map (maybe 1 boolSign . flip lookupVar solver) vars where
            boolSign True = -1
            boolSign False = 1
        return solution

Let's **test it out first** on the $N=2$ matrix.

In [12]:
-- What does it tell us about the N=2 problem?
findNiceBasis (tMatrixN 2 (productBasisN 2))

[-1,-1,-1,-1]

The return value is a list of sign factors to apply to each ket in order to produce a matrix of all -t elements.  Since the $\mathbf T$ matrix for $N=2$ is already all negative, no negations are necessary. (though interestingly, the code produces the equivalent solution to _negate everying!_)

That test had no positive elements in the matrix, so it doesn't instill much confidence. Thus, for our next test, **let's flip the third basis vector and see if it flips it back:**

In [13]:
do
    let basis = productBasisN 2
    let basis' = let [b0,b1] = basis ! 2
                 in basis // [(2, [b1,b0])]
    findNiceBasis (tMatrixN 2 basis')

[-1,-1,1,-1]

Here we see it **did flip the third basis ket back**.

As a final test, let's be evil and give it something *impossible* by **putting a positive element on the main diagonal.**

In [14]:
do
    let ((_:m0'):m') = tMatrixN 2 (productBasisN 2)
    let matrix = (1:m0'):m'
    findNiceBasis matrix

user error (mzero)

This (unfortunately plain) error message means that the SAT solver determined that our boolean expression was unsatisfiable. Excellent! Now for the real deal:

**How can we choose signs to make the $N=3$ matrix negative?**

In [15]:
findNiceBasis (tMatrixN 3 hw4Basis)

user error (mzero)

Huh, imagine that! **It would appear that there is no assignment of basis kets for the $N=3$ problem such that all jumping interaction terms are negative!** Well, at least we tried!

The same also happens for $N=4$:

In [16]:
findNiceBasis (tMatrixN 4 (productBasisN 4))

user error (mzero)

## My 6 terms

In [17]:
myElems = [(4,c) | c <- [4..9]]

do
    let tMatrix = tMatrixN 3 hw4Basis
    let uMatrix = uMatrixN 3 hw4Basis
    forM_ myElems $ \(r,c) -> do
        let tValue = tMatrix !! (r-1) !! (c-1)
        let uValue = uMatrix !! (r-1) !! (c-1)
        putStrLn $ "H_" <> show r <> show c
                 <> " = " <> show tValue <> "t"
                 <> " + " <> show uValue <> "u"

H_44 = 0t + 1u
H_45 = -1t + 0u
H_46 = 1t + 0u
H_47 = 0t + 0u
H_48 = 0t + 0u
H_49 = 0t + 0u